In [81]:
# Process BES Panel for Wave 8

## WORK IN PROGRESS - run this on all updated BES waves

# Assign types to each column variables (mostly automatic - but there's room for manual editing)
# Modifies categories (harmoning options, harmonising order, dropping weasel terms)
# Creates:
# var_type (types for column variables)
# BES_numeric (fixed version with only useful columns all transformed into numeric values)
    # non-numeric categorical values -> index number of category
    # floats -> float
    # numeric categories (i.e. income intervals/number of children) -> numeric
# cat_dictionary
    # so you can get the text categories back from the index values as and when it's necessary
# BES_non_numeric
    # basically everything else! (mostly meta-data - weights)

# LIVE: Run all cells -> ~20s

In [7]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gc
import re

from itertools import cycle
from IPython.display import display
import pickle, os

import seaborn as sns

BES_data_folder = "../BES_analysis_data/"
BES_code_folder = "../BES_analysis_code/"


In [8]:
%%time

dataset = "BES2017_W13_v1.0.dta"
wave = "W13_only"

BES_Panel = pd.read_stata(BES_data_folder+dataset)

print( BES_Panel.shape )

(31197, 312)
Wall time: 3.05 s


In [84]:
%%time
## HELPER FUNCTIONS / REPLACEMENT VALUE DICTIONARIES


# changing the order of some sets of categories
change_cat_dict = {"Bad time to buy|Good time to buy|Neither good nor bad time to buy|Don't know": ["Bad time to buy",
                                                                                                    "Neither good nor bad time to buy",
                                                                                                    "Good time to buy",
                                                                                                    "Don't know"],
                   "Larger|Smaller|About the same|Don't know": ["Larger", "About the same", "Smaller","Don't know"],
                   "Yes|No|99.0":       ['No', 'Yes', '99.0'],
                   "Yes|No|Don't know": ['No', 'Yes', "Don't know"],
                   "Yes|No" :           ['No', 'Yes'],
                   "Yes|No|Did not vote|Don't know" : ["No","Yes","Did not vote","Don't know"],
                   "Yes, voted|No, did not vote|Donâ??t know" : ['No, did not vote', 'Yes, voted', 'Donâ??t know'],
                   "Yes, voted|No, did not vote|Don?t know"   : ['No, did not vote', 'Yes, voted', 'Don?t know'],                   
                   "I would/will not vote|Leave the EU|Stay/remain in the EU|Don't know": ['Stay/remain in the EU',
                                                                                           'Leave the EU', 'I would/will not vote', "Don't know"],
                   "Mainly leave|Mainly remain|Fairly evenly divided|Don't know": ["Mainly remain",
                                                                                   "Fairly evenly divided", "Mainly leave", "Don't know"],
                   "An individual share in a company|A portfolio of different company shares|The risk is the same|Don't know|Prefer not to say":
                       ['An individual share in a company', 'The risk is the same', 'A portfolio of different company shares',"Prefer not to say","Don't know"],
                   'An individual share in a company|A portfolio of different company shares|The risk is the same|Don\x92t know|Prefer not to say':
                       ['An individual share in a company', 'The risk is the same', 'A portfolio of different company shares',"Prefer not to say","Don\x92t know"],
                   "No, I have never been a member|Yes, I am a member of a party|I am not a member now but I used to be|Don't know":
                       ['No, I have never been a member', 'I am not a member now but I used to be', 'Yes, I am a member of a party', "Don't know"],
                   "Never or practically never|Less often than once a year|Less often but at least once a year|Less often but at least twice a year|Less often but at least once a month|Less often but at least once in two weeks|Once a week or more|Varies too much to say|I am not religious|Don't know":
                       ['I am not religious', 'Never or practically never', 'Less often than once a year',
                        'Less often but at least once a year', 'Less often but at least twice a year',
                        'Less often but at least once a month', 'Less often but at least once in two weeks',
                        'Once a week or more', "Varies too much to say","Don't know"],
                   "under Â£5,000 per year|Â£5,000 to Â£9,999 per year|Â£10,000 to Â£14,999 per year|Â£15,000 to Â£19,999 per year|Â£20,000 to Â£24,999 per year|Â£25,000 to Â£29,999 per year|Â£30,000 to Â£34,999 per year|Â£35,000 to Â£39,999 per year|Â£40,000 to Â£44,999 per year|Â£45,000 to Â£49,999 per year|Â£50,000 to Â£59,999 per year|Â£60,000 to Â£69,999 per year|Â£70,000 to Â£99,999 per year|Â£100,000 to Â£149,999 per year|Â£150,000 and over|Don't know|Prefer not to answer":
                       ["under Â£5,000 per year","Â£5,000 to Â£9,999 per year","Â£10,000 to Â£14,999 per year","Â£15,000 to Â£19,999 per year",
                        "Â£20,000 to Â£24,999 per year","Â£25,000 to Â£29,999 per year","Â£30,000 to Â£34,999 per year",
                        "Â£35,000 to Â£39,999 per year","Â£40,000 to Â£44,999 per year","Â£45,000 to Â£49,999 per year",
                        "Â£50,000 to Â£59,999 per year","Â£60,000 to Â£69,999 per year","Â£70,000 to Â£99,999 per year",
                        "Â£100,000 to Â£149,999 per year","Â£150,000 and over","Prefer not to answer","Don't know"],
                   "1|2|3|4|5|6|7|8 or more|Don't know|Prefer not to say":
                       ["1","2","3","4","5","6","7","8 or more","Prefer not to say","Don't know"],
                   "The Yes side|The No side|Neither|Don't know":
                       ["The Yes side","Neither","The No side","Don't know"] # is this ordinal - meh?
                   
                  }

# Big set of actual answers **I interpet** as non-answers (and set to NaN)
# REALLY MERITS RECHECKING WHAT THE IMPACT OF THIS IS!
Weasel_answers = ["Don't know", 'Do\x92t know', 'Dont know', 'Donât know', 'Don??t know',
                  "Prefer not to say", "Prefer not to answer", "Refused", "Unknown",
                  "Neither", "Other", "I would/will not vote", "Will not vote",
                  "I would not vote", "It depends", "Other",
                  "Don’t follow politics on Facebook", "Don't follow politics on twitter",
                  "Yes, other", "Haven't thought about it",
                  "There wasn't a local election in my area", "No, haven't received it",
                  "I don't know what was negotiated", "I never received a response",
                  "There are not local elections in my area", "Can't remember",
                  "Varies too much to say", "Will not state a choice",
                  "All leaders equally good", "They are not eligible to vote",
                  "There are not local elections in my area", "Both/neither",
                  "Did not vote","Can't remember"]

# BES codes for NaN
Weasel_number_answers = [ "9999.0", "997.0", "222.0", "99.0", "0.0" ]

# non-answer answers
Weasel_set = set(Weasel_answers) # gets rid of duplicates!


## define 'de_Weasel' function to remove Weasel Words from lists of options
## ie. "Yes|No|Don't know" -> "Yes|No"

# Weasel_answers = ["Don't know", 'Don?t know', 'Donâ??t know', 'Do\x92t know', 'Dont know', 'Donât know',
#                   "Prefer not to say", "Prefer not to answer", "Refused", "Unknown", "Neither", "Other",
#                   "I would/will not vote", "Will not vote", "No - not decided", "I would not vote", "It depends",
#                   "Other", "Don’t follow politics on Facebook", "Don't follow politics on twitter", "9999.0", "997.0",
#                   "222.0", "Yes, other", "Haven't thought about it", "There wasn't a local election in my area",
#                   "No, haven't received it", "I don't know what was negotiated", "I never received a response",
#                   "There are not local elections in my area", "Can't remember", "Varies too much to say" ]

# # non-answer answers
# Weasel_set = set(Weasel_answers) # gets rid of duplicates!

# remove weasel phrases
def de_weasel(ques): 

    return "|".join( [x for x in ques.split("|") if x not in Weasel_answers] )

# reorder categories
def re_order(ques):
    if ques in change_cat_dict.keys():
        return "|".join( change_cat_dict[ques] )
    else:
        return ques

def de_num_el(el):
    if el.isdigit():
        el = "%.1f" % int( el )
    return el

def de_number(ques):
    return "|".join( [de_num_el(x) for x in ques.split("|")] )

def de_num(ques):
    return [de_num_el(x) for x in ques]

def floatable(flt):
    try:
        float(flt)
        return True
    except:
        return False

# Weasel_number_answers
# Remove 'weasel' numbers
# but only if they are the last element
# or not the last element, but the next is not a number
# to avoid catching parts of sequential numerical categories
def de_weasel_numbers(ques):
    el_list = ques.split("|")
    el_list_len = len(el_list)
    remove_list = []
    for el_pos in range( 0, el_list_len ):
        if el_list[el_pos] in Weasel_number_answers:
            # last element, or not last element but next element is a not a number
            if el_pos==(el_list_len-1) or not floatable(el_list[el_pos+1]):
                remove_list.append(el_list[el_pos])

    return "|".join( [x for x in el_list if x not in remove_list] )


# version to act directly on cat.categories array
def de_weasel_nums(el_list):

    el_list_len = len(el_list)
    remove_list = []
    for el_pos in range( 0, el_list_len ):
        if el_list[el_pos] in Weasel_number_answers:
            # last element, or not last element but next element is a not a number
            if el_pos==(el_list_len-1) or not floatable(el_list[el_pos+1]):
                remove_list.append(el_list[el_pos])

    return remove_list

# s.cat.rename_categories([1,2,3])
# EUContactRemainConW8|EUContactRemainLabW8|EUContactRemainLDW8|
# EUContactRemainSNPW8|EUContactRemainPCW8|EUContactRemainUKIPW8|
# EUContactRemainGreenW8|EUContactRemainOthW8|EUContactRemainNoneW8|
# EUContactRemainDKW8|EUContactLeaveConW8|EUContactLeaveLabW8|
# EUContactLeaveLDW8|EUContactLeaveSNPW8|EUContactLeavePCW8|
# EUContactLeaveUKIPW8|EUContactLeaveGreenW8|EUContactLeaveOthW8|
# EUContactLeaveNoneW8|EUContactLeaveDKW8

# pattern match "EUContact*****W8"
# debateOneWatchW8|debateTwoWatchW8

# "1.0|2.0|99.0" -> 

# euRefVoteSqueezeW7 "Will not vote|Yes - Leave|Yes - Remain|No - not decided"
#    -> Stay/remain in the EU|Leave the EU|I would/will not vote|Don't know
#    HMM - RENAME AND REORDER!

# miieuW7
# "Issue stated|Nothing|Don't know" -> "Issue stated|None|Don't know"
# MIIEUW8
# "1.0|Nothing|Don't know" -> "Issue stated|None|Don't know"
# partyIdEUW7|partyIdEUW8
# "Mainly leave|Mainly remain|Fairly evenly split|Don't know" -> "Mainly remain|Fairly evenly divided|Mainly leave|Don't know"
#    HMM - RENAME AND REORDER!

# 1. campaignVisionYesW3|campaignVisionNoW3, govtNatSecuritySuccessW4
# Very unsuccessful|Fairly unsuccessful|Neither successful nor unsuccessful|Fairly successful|Very successful|Don't know
# Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful|Don't know

# Fairly <-> Somewhat

# 2. euroTurnoutW1, scotReferendumTurnoutW1|scotReferendumTurnoutW2|welshTurnoutW7|scotTurnoutW7, turnoutUKGeneralW1|turnoutUKGeneralW2|turnoutUKGeneralW3|turnoutUKGeneralW4|turnoutUKGeneralW5|euRefTurnoutW7|euRefTurnoutW8
# Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote|Don't know
# Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote|Don't know
# There are not local elections in my area
    #|Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote|Don't know
# Very unlikely that I will vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I will vote|Don't know

# "Very unlikely that I vote", "Very unlikely that I would vote" ->  "Very unlikely that I will vote" 

rename_cat_dict = {"North East|North West": [ "No", "Yes" ],
                   "1.0|2.0|99.0": ["No", "Yes", "99.0"],
                   "Will not vote|Yes - Leave|Yes - Remain|No - not decided": ['I would/will not vote', 'Leave the EU',
                                                                               'Stay/remain in the EU', "Don't know"],
                   "Issue stated|Nothing|Don't know":  ['Issue stated', 'None', "Don't know"],
                   "1.0|Nothing|Don't know":           ['Issue stated', 'None', "Don't know"],
                   "a|b|C1|C2|d|e|Refused|Unknown" : ['A', 'B', 'C1', 'C2', 'D', 'E', 'Refused', 'Unknown'],
                   "Mainly leave|Mainly remain|Fairly evenly split|Don't know": ['Mainly leave',
                                                                                 'Mainly remain', 'Fairly evenly divided', "Don't know"],
                   "Very unsuccessful|Somewhat unsuccessful|Neither successful or unsuccessful|Somewhat successful|Very successful|Don't know": ['Very unsuccessful',
                        'Fairly unsuccessful', 'Neither successful nor unsuccessful', 'Fairly successful', 'Very successful', "Don't know"],
                   "Very unlikely that I vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I vote|Don't know":
                       ['Very unlikely that I will vote',
                     'Fairly unlikely', 'Neither likely nor unlikely', 'Fairly likely', 'Very likely that I will vote', "Don't know"],
                   "Very unlikely that I would vote|Fairly unlikely|Neither likely nor unlikely|Fairly likely|Very likely that I would vote|Don't know":
                       ['Very unlikely that I will vote', 'Fairly unlikely',
                        'Neither likely nor unlikely', 'Fairly likely', 'Very likely that I will vote', "Don't know"],
                   "No, did not vote|Yes, voted|3.0|4.0|5.0|Don't know":   
                       ["Very unlikely that I will vote", "Fairly unlikely",
                        "Neither likely nor unlikely", "Fairly likely", "Very likely that I will vote", "Don't know"], #londonTurnoutW7
                   "No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal|Yes - Roman Catholic|Yes - Presbyterian/Church of Scotland|Yes - Methodist|Yes - Baptist|Yes - United Reformed Church|Yes - Free Presbyterian|Yes - Brethren|Yes - Judaism|Yes - Hinduism|Yes - Islam|Yes - Sikhism|Yes - Buddhism|Yes - Other|Prefer not to say|Yes â€“ Orthodox Christian|Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur|Yes - Evangelical â€“ independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)":
                       ["No, I do not regard myself as belonging to any particular religion.","Yes - Church of England/Anglican/Episcopal",
                        "Yes - Roman Catholic","Yes - Presbyterian/Church of Scotland","Yes - Methodist","Yes - Baptist",
                        "Yes - United Reformed Church","Yes - Free Presbyterian","Yes - Brethren","Yes - Judaism","Yes - Hinduism",
                        "Yes - Islam","Yes - Sikhism","Yes - Buddhism","Yes - Other","Prefer not to say","Yes - Orthodox Christian",
                        "Yes - Pentecostal","Yes - Evangelical /independent/non-denominational"], #xprofile_religionW10
                   "Own â€“ outright|Own â€“ with a mortgage|Own (part-own) â€“ through shared ownership scheme (i.e. pay part mortgage, part rent)|Rent â€“ from a private landlord|Rent â€“ from my local authority|Rent â€“ from a housing association|Neither â€“ I live with my parents, family or friends but pay some rent to them|Neither â€“ I live rent-free with my parents, family or friends|Other":
                       ["Own outright","Own with a mortgage","Own (part-own) through shared ownership scheme (i.e. pay part mortgage, part rent)",
                        "Rent from a private landlord","Rent from my local authority","Rent from a housing association",
                        "Neither I live with my parents, family or friends but pay some rent to them",
                        "Neither I live rent-free with my parents, family or friends","Other"], #xprofile_house_tenureW10
                   "I voted 'No' (Scotland should not be an independent country)|I voted 'Yes' (Scotland should be an independent country)|111.0|Don't know":
                       ["No","Yes","Did not vote","Don't know"],
                   "Voted Yes|Voted No|Did not vote|Can't remember":
                       ["Yes","No","Did not vote","Don't know"]
                   
                   
                   }



def re_name(ques):
    if ques in rename_cat_dict.keys():
        return "|".join( rename_cat_dict[ques] )
    else:
        return ques

## COLUMNS THAT EITHER LACK ALL DATA OR HAVE ACTUAL ERRORS
# check back on these periodically - one assumes they will get fixed!
# maybe tell them about them so that they can?
ignore_list = ['approveEUW2',
               'whichPartiesHelped_99W6',
               'partyContactGrnW1',
               'partyContactGrnW2',
               'partyContactGrnW3',
               'reasonNotRegistered_noneW2',               
               'reasonNotRegistered_noneW3',
               'reasonNotRegistered_noneW4',
               'reasonNotRegistered_noneW6',
               'reasonNotRegistered_noneW7',
               'reasonNotRegistered_noneW8',
               'reasonNotRegistered_none',
               'partyContactSNPW1',
               'partyContactSNPW2',
               'changeIssue1W9',
               'conLeaderLikeW9',
               "locusControlW9",
               "generalElecCertaintyW1", # wave 10 forwards
               "generalElecCertaintyW2",
               "generalElecCertaintyW3",
               "londonMayorVoteW7",
               "fatherNumEmployeesW4",
               "motherNumEmployeesW4",
               "selfNumEmployeesW6W7W8W9",
               "selfNumEmployeesLastW6W7W8W9"
              ]

#- approveEUW2 'Strongly disapprove|Disapprove|Don't know' - should be "approve|disapprove|don't know"??? NOT SURE (distribution weird)
#- whichPartiesHelped_99W6 - answer set = ["No"]
#- partyContactGrnW1 ... reasonNotRegistered_noneW8 answer set = ["No", "Don't know"]
# -partyContactSNPW1, partyContactSNPW2 - answer set = ["Don't know"]
# -changeIssue1W9|conLeaderLikeW9|locusControlW9 - answer set = ["No formal qualifications"]

## define 'prune' function to prune wave indicators and return question stubs
## ie. "ptvConW1|ptvLabW1" -> "ptvCon|ptvLab"

def prune(x):
    
    y= []
    for el in x.split("|"):
        match_attempt = re.match('(\w*?)_?(W[0-9]+)+' , el )   
        if match_attempt:
            el = match_attempt.groups()[0]
        y.append(el)
    # should we ditch identical repeats?
    # return "|".join(set(y)) NEEDS TO BE TESTED
    return "|".join(y)

               
def prune2(x):
    
    y= []
    for el in x.split("|"):
        # fgdfhfghg_5, fgdfhfghg_4, fgdfhfghg_3 -> fgdfhfghg
        # problem - indicator variables fgdfhfghg_99, fgdfhfghg_111 really are different!
        # solution - leave them distinct
        indicator_variable = re.match('(\w*?)_?(99|111)' , el )       
        match_attempt = re.match('(\w*?)_?[0-9]+' , el )   
        if (not indicator_variable) and (match_attempt):
            el = match_attempt.groups()[0]
        y.append(el)
    # should we ditch identical repeats?
    # return "|".join(set(y)) NEEDS TO BE TESTED
    return "|".join(y)
#variable_categories

Wall time: 0 ns


In [85]:
## dataset specific issues
# (i.e. probably what I should have done all along!)

# "BES2017_W13_v1.0.dta"
if (dataset == "BES2017_W13_v1.0.dta"):
    BES_Panel["headHouseholdPast"] = \
        BES_Panel["headHouseholdPast"].astype("category").cat.rename_categories(["My father",
                                                                             "My mother",
                                                                             "Someone else",
                                                                             "No one in my house worked",
                                                                             "Don't know"])
        
    BES_Panel["fatherNumEmployees"] = \
        BES_Panel["fatherNumEmployees"].astype("category").cat.rename_categories(["1 to 24 employees",
                                                                             "25 to 499 employees",
                                                                             "500 or more employees",
                                                                             "Don't know"])        
    
    BES_Panel["motherNumEmployees"] = \
        BES_Panel["motherNumEmployees"].astype("category").cat.rename_categories(["1 to 24 employees",
                                                                             "25 to 499 employees",
                                                                             "500 or more employees",
                                                                             "Don't know"])   

In [86]:
variable_categories = pd.read_csv("question_categories_correct.csv", encoding = "ISO-8859-1")
variable_categories.drop('Unnamed: 0', axis=1,inplace=True)
# flipping list
var_cat_dict = dict()
for typ in [0,1,2,3,4,5,6,7]:
    # 
    e = variable_categories[variable_categories.type==typ]["column_name"].values
    var_cat_dict[typ] = [item for sublist in [i.split("|") for i in e] for item in sublist]
    var_cat_dict[typ] = [item for item in var_cat_dict[typ] if item not in ignore_list]
    
# dictionary comprehension to prune column-names to wave non-specific stubs
# list(set()) gets rid of repetitions
var_cat_dict_pruned   = {k: list(set([prune(x)  for x in v])) for k, v in var_cat_dict.items()}
var_cat_dict_pruned_2 = {k: list(set([prune2(x) for x in v])) for k, v in var_cat_dict_pruned.items()}

In [87]:
var_type = pd.DataFrame(columns = ['type'] )

missing_col_names = []

for col in BES_Panel.columns:
    dt =  BES_Panel[col].dtype.name # data type
    not_found = False
    
    if col in ignore_list: # exclude values from ignore_list
        var_type.loc[col] = -2
        
    elif (col == "id"): # id
         var_type.loc[col] = -5

    elif (dt == 'object'): # text
        var_type.loc[col] = -4

    elif ("datetime" in dt): # datetime
        var_type.loc[col] = -3           

    else:
        not_found = True

        for typ in [0,1,2,3,4,5,6,7]: 
            if prune2( prune(col) ) in var_cat_dict_pruned_2[typ]:
                var_type.loc[col] = typ
                not_found = False

    if not_found == True:
        var_type.loc[col] = -1
        #print("what's up with this? " + col, prune2( prune(col) ) )
        missing_col_names.append(col)
var_type["type"] = var_type["type"].astype("int8")
# missing_col_names

In [88]:
#######################################################################
## MANUAL FIXING OF COLUMN NAMES THAT DON'T ALREADY HAVE A TYPE
## Only works for category type variables (you may need to manually drop non-category columns!)
## Basically, does its best to automatically guess the matching types (based on the near match of the {answer set})
## Appends them to the a copy of "questions_category_correct" named 'question_categories_correct_updatesneeded!.csv'
## Then I'm afraid you have to look at them, fix them if needed, and then run the code in the cell below again
manual_fixing = False
missing_col_names_cat_only = []
if manual_fixing:
    print("#######################################")
    print("CATEGORIES")
    print("#######################################")
    print("\n")
    for col in missing_col_names:
        if BES_Panel[col].dtypes.name == 'category':
            print(col)
            missing_col_names_cat_only.append(col)
    print("\n")
    print("\n")

    print("#######################################")
    print("NON CATEGORIES")
    print("#######################################")
    print("\n")
    for col in missing_col_names:
        if BES_Panel[col].dtypes.name != 'category':
            print(col)
    print("\n")
    print("\n")
    ## if this only shows "weight" variables, it's fine - set manual_fixing to false and ignore!

In [89]:
if manual_fixing:

    updated_variable_categories = variable_categories.copy()

    # question	frequency	question_length	question_options	column_name	type

    for i in missing_col_names_cat_only:
        str_list = [ str(cat) for cat in BES_Panel[i].cat.categories ]
        joined_list = "|".join(str_list)
        match  = (joined_list == updated_variable_categories["question"])
        # print(i, " : " , "|".join(str_list ), " : ", len(str_list) )
        if match.any(): # answer set already in records
            index = updated_variable_categories[match].index
            if len(index)>1: # answer set ("question") index should be unique!
                raise ValueError('answer set ("question") index should be unique!')

            # add column name and increase frequency
            updated_variable_categories.loc[index,"frequency"] = updated_variable_categories.loc[index,"frequency"]+1
            current_list_col_names = updated_variable_categories.loc[index,"column_name"].values[0].split("|")
            current_list_col_names.append(i)
            updated_variable_categories.loc[index,"column_name"] = "|".join( current_list_col_names )
        else: # answer set not already in records - add new line to dataframe
            df = pd.DataFrame([],  columns = updated_variable_categories.columns )

            # no need to add index
            # updated_variable_categories.shape[0], 
            df.loc[0] = [joined_list,
                         1,
                         len(joined_list),
                         len(str_list),
                         i,-1]
            updated_variable_categories = updated_variable_categories.append(df, ignore_index=True)

    variable_categories = updated_variable_categories
    updated_variable_categories.to_csv("question_categories_correct_updatesneeded!.csv", encoding = "ISO-8859-1")
    
    # rerun after updating list!
    
    #variable_categories = pd.read_csv("question_categories_correct.csv", encoding = "ISO-8859-1")

    # flipping list
    var_cat_dict = dict()
    for typ in [0,1,2,3,4,5,6,7]:
        e = variable_categories[variable_categories.type==typ]["column_name"].values
        var_cat_dict[typ] = [item for sublist in [i.split("|") for i in e] for item in sublist]

    # dictionary comprehension to prune column-names to wave non-specific stubs
    # list(set()) gets rid of repetitions
    var_cat_dict_pruned   = {k: list(set([prune(x)  for x in v])) for k, v in var_cat_dict.items()}
    var_cat_dict_pruned_2 = {k: list(set([prune2(x) for x in v])) for k, v in var_cat_dict_pruned.items()}        
        
    var_type = pd.DataFrame(columns = ['type'] )
    missing_col_names = []

    for col in BES_Panel.columns:
        dt =  BES_Panel[col].dtype.name # data type
        not_found = False
        if col in ignore_list: # exclude values from ignore_list
            var_type.loc[col] = -2
            
        elif (col == "id"): # id
            var_type.loc[col] = -5
            
        elif (dt == 'object'): # text
            var_type.loc[col] = -4

        elif ("datetime" in dt): # datetime
            var_type.loc[col] = -3          

        else:
            not_found = True

            for typ in [0,1,2,3,4,5,6,7]:
                if prune2( prune(col) ) in var_cat_dict_pruned_2[typ]:
                    var_type.loc[col] = typ
                    not_found = False

        if not_found == True:
            var_type.loc[col] = -1
            missing_col_names.append(col)
#             raise ValueError('Values still missing second time around! ', col)
    var_type["type"] = var_type["type"].astype("int8")
    display([x for x in zip(missing_col_names, BES_Panel[missing_col_names].dtypes)])

In [90]:
# BES_Panel[ ['referendumrecall', 'scotRefVoteW4_'] ]

In [91]:
# %%time

# ditch ignore_list values
# ditch indicator values

num_cols     = BES_Panel.columns[ (var_type["type"].apply( lambda x: x in [0,1,2,3,5,6,7] )).values ]
non_num_cols = BES_Panel.columns[ (var_type["type"].apply( lambda x: x in [-5,-4,-3,-1 ] )).values ]

BES_numeric     = BES_Panel[num_cols].copy()


pos = 0
for col in BES_numeric:
    pos = pos + 1
#     print(col)
    
    if col not in var_type["type"].index:
        print(col, " not in var_type")
        continue
    typ = var_type["type"][col]
    
    if (typ==0) | (typ==7):
        continue
    
    # force all category elements into strings
    BES_numeric[col].cat.rename_categories( BES_numeric[col].cat.categories.map(str), inplace=True )
    
    # rename categories
    
    join_list = "|".join( BES_numeric[col].cat.categories )
    if join_list in rename_cat_dict.keys():
        BES_numeric[col].cat.rename_categories(  rename_cat_dict[join_list], inplace=True )
    # update join_list!
    
    # reorder categories
    join_list = "|".join( BES_numeric[col].cat.categories )
    if join_list in change_cat_dict.keys():
        BES_numeric[col].cat.reorder_categories( change_cat_dict[join_list], inplace=True )    
    
    # de_weasel numbers
    BES_numeric[col].cat.remove_categories( de_weasel_nums( BES_numeric[col].cat.categories ), inplace=True )
    
    # set all digits to floating point format, one decimal place
    BES_numeric[col].cat.rename_categories( de_num( BES_numeric[col].cat.categories ), inplace=True )
                                          
    # de_weasel                                   
    BES_numeric[col].cat.remove_categories( BES_numeric[col].cat.categories.intersection(Weasel_set), inplace=True )


In [92]:
# types
# -5    id
# -4    text
# -3    datetimes
# -2 - ignore_list
# -1 - anything not a category or one of the below floats -> should all be weights! (wave indicators/campaign day indicators)
# 0 - personality measures (in steps of .5?), personality_agreeableness ...etc, riskScale
# 1 - linear category, just use  (some made linear by dropping "Weasel_answers")
# 2 - categories need to be modified - order changed
# 3 - set of non-ordered options
# 4 - indirect variables - did someone fill something in in the free text box or not?
# 5 - categories need to modified - things removed
# 6 - categories are integers - should maybe be transformed directly into numbers (mostly "how much money do people need minimum/well off"?)
# 7 - pano, mapNames



# [-5, -4, -3, -2, -1, 4] -> meta list
# [0, 1, 2, 3, 5, 6, 7] -> 

# ordinal: 0, 1, 2, 5, 6
# non-ordinal: 3, 7

# load question_categories_correct.csv
# sanity check by type!
# turn into list of variables by type
# 1, 5 handled the same way -> cat.codes
# 6 -> int()
# 4 ignored
# 3 ignored for now (-> vectorized?)
# 2 direct modification

In [93]:
gc.collect()

7629

In [94]:
# save category data
cat_dictionary = {}

for col in BES_numeric.columns:
#     print(col)
    if var_type["type"][col] in [1, 2, 3, 5]: # not just cat, but one not already numerical!
        cat_dictionary[col] = BES_numeric[col].cat.categories


In [95]:
# %%time

# turn categories into numbers

for col in BES_numeric:
    
    typ = var_type["type"][col]
    
    if (typ == 0): # not necessarily already float now!
        BES_numeric[col] = BES_numeric[col].astype('float64')
        
    elif (typ==1) | (typ==2) | (typ==5): # more or less ordinal, replace string categories with 
        BES_numeric[col] = BES_numeric[col].cat.codes
        BES_numeric[col] = BES_numeric[col].astype('float64')
     
    elif (typ==3): # categporical not ordinal
        BES_numeric[col] = BES_numeric[col].cat.codes
        BES_numeric[col] = BES_numeric[col].astype('float64')
        
    elif (typ==6): # categories are integers - better to translate directly
        BES_numeric[col] = BES_numeric[col].astype('float64')
        
    elif (typ==7): # integers - better to translate directly
        BES_numeric[col] = BES_numeric[col].astype('float64')        
        
BES_numeric.replace(-1,np.nan, inplace=True)

In [96]:
gc.collect()

2058

In [97]:
BES_numeric.replace(-1,np.nan, inplace=True)

In [98]:
# BES_numerics_only = BES_numeric.drop( BES_numeric.columns[~( (var_type["type"]==0) |
#                                                              (var_type["type"]==1) |
#                                                              (var_type["type"]==2) |
#                                                              (var_type["type"]==5) |
#                                                              (var_type["type"]==6) ) ], axis=1 )

# BES_numerics_only.replace(-1,np.nan, inplace=True)
# # gender only column that has no nan -> still an int

In [99]:
# BES_all_categories = BES_numeric.drop( BES_numeric.columns[~( (var_type["type"]==0) |
#                                                              (var_type["type"]==1) |
#                                                              (var_type["type"]==2) |
#                                                              (var_type["type"]==3) |
#                                                              (var_type["type"]==5) |
#                                                              (var_type["type"]==6) ) ], axis=1 )

# BES_all_categories.replace(-1,np.nan, inplace=True)
# # gender only column that has no nan -> still an int

In [100]:
# BES_numerics_only = BES_all_categories

In [101]:
# BES_num_and_cat.to_stata( BES_data_folder+"BESW8num_and_cat.hdf" )

In [102]:
wave = "W13_only"

In [103]:
fname = BES_data_folder+"cat_dictionary"+wave+".pkl"
with open(fname, "wb") as f:
    pickle.dump( cat_dictionary, f )

In [104]:
BES_non_numeric = BES_Panel[non_num_cols].copy()
BES_non_numeric.to_hdf( BES_data_folder+"BESnon_numeric"+wave+".hdf", "BESnon_numeric"+wave )


In [105]:
BES_numeric.to_hdf( BES_data_folder+"BESnumeric"+wave+".hdf", "BESnumeric"+wave )

In [106]:
var_type.to_hdf( BES_data_folder+"var_type"+wave+".hdf", "var_type" )

In [107]:
# fname = BES_data_folder+"cat_dictionary"+".pkl"
# with open(fname, "wb") as f:
#     pickle.dump( cat_dictionary, f )
    
# BES_non_numeric = BES_Panel[non_num_cols].copy()
# BES_non_numeric.to_hdf( BES_data_folder+"BESW8non_numeric.hdf", "BESW8non_numeric" )

# BES_numeric.to_hdf( BES_data_folder+"BESW8numeric.hdf", "BESW8numeric" )

# var_type.to_hdf( BES_data_folder+"var_type.hdf", "var_type" )

In [108]:
var_type["type"].value_counts()

 1    254
 3     30
 5      8
-1      6
 0      5
-4      2
-3      2
 7      1
 4      1
 2      1
 6      1
-5      1
Name: type, dtype: int64

In [109]:
# import sys
# for var, obj in locals().items():
#     if sys.getsizeof(obj) >100000:
#         print(var, sys.getsizeof(obj))

In [110]:
BES_numeric.shape

(31197, 300)

In [111]:
# TEST CODE
# Checking to see if different column names have same question set but are assigned different types

In [112]:
# Test to see if pruning "_wave" leads to differing variable categories
for key in var_cat_dict_pruned.keys():
    for val in var_cat_dict_pruned[key]:
        for other_key in var_cat_dict_pruned.keys():
            if key==other_key: # don't check same category
                continue
            if val in var_cat_dict_pruned[other_key]:
                print("problem: {0} {1} {2}".format(key, val, other_key) )
            
# mixing 1 and 5 is fine, because 5 (mis-ordered ordinal) is turned into 1 (correctly ordered ordinal)

problem: 1 euRefVoteSqueeze 5
problem: 5 euRefVoteSqueeze 1


In [113]:
# Test to see if further pruning "_num" leads to differing variable categories
for key in var_cat_dict_pruned_2.keys():
    for val in var_cat_dict_pruned_2[key]:
        for other_key in var_cat_dict_pruned_2.keys():
            if key==other_key: # don't check same category
                continue
            if val in var_cat_dict_pruned_2[other_key]:
                print("problem: {0} {1} {2}".format(key, val, other_key) )
            
                

problem: 1 euRefVoteSqueeze 5
problem: 1 euID 5
problem: 1 finlit 2
problem: 2 finlit 1
problem: 5 euRefVoteSqueeze 1
problem: 5 euID 1


In [114]:
# pano, mapNames
# weights
# personality
# datetimes -> remove on type
# text -> remove on type

# weights, datetimes, text -> saved separately

In [115]:
# (BES_Panel[ 'CampaignDayW5' ]-19).abs()

In [116]:
# W5_cats_df.corrwith( (BES_Panel[ 'CampaignDayW5' ]-19).abs() ).sort_values()

In [117]:
# gc.collect()

In [118]:
# corr_df["absolute corr"].mean()

In [119]:
# sample_size = W5_cats_df.notnull().apply(lambda x: x& BES_Panel[ 'CampaignDayW5' ].notnull()).sum()
# corr = W5_cats_df.corrwith(BES_Panel[ 'CampaignDayW5' ])


# corr_df = pd.DataFrame(corr, columns= ["corr"])
# corr_df["sample_size"] = sample_size
# corr_df["absolute corr"] = corr.abs()
# corr_df[corr_df["sample_size"]>2000].sort_values(by="absolute corr", ascending=False).head(21)

In [120]:
# # BES_Panel[ 'CampaignDayW5' ].corr(W5_cats_df)
# W5_cats_df.corrwith(BES_Panel[ 'CampaignDayW5' ]).sort_values().hist(bins=100)

In [121]:
# # 283 with "W5" in
# # 248 with "W5" in AND category

# W5cats = [x for x in BES_Panel.columns if ("W5" in x) & (BES_Panel[x].dtype.name =='category')]


In [122]:
# W5_cats_df = BES_Panel[W5cats].apply(lambda x : x.cat.codes).replace(-1,np.nan)

In [123]:
# BES_Panel["selfNumEmployeesLastW6W7W8W9"].value_counts()

In [124]:
# BES_Panel["londonTurnoutW7"].cat.categories